In [19]:
import time
from datetime import datetime
import json
import hmac
import hashlib
import requests
from urllib.parse import urljoin, urlencode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

API_KEY = '*****************'
SECRET_KEY = '******************'
BASE_URL = 'https://api.mexc.com'
headers = {
    'X-MEXC-APIKEY': API_KEY
}

# API Functions

In [36]:
def get_current_price():
    PATH = '/api/v3/avgPrice'
    params = {
        'symbol': 'TAOUSDT'
    }

    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, headers=headers, params=params)
    return float(r.json()['price'])

def sell_tao(quantity, price):
    PATH = '/api/v3/order'
    timestamp = int(time.time() * 1000)
    params = {
      'symbol': 'TAOUSDT',
      'side': 'SELL',
      'type': 'LIMIT',
      'quantity': quantity,
      'price': price,
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.post(url, headers=headers, params=params)
    if r.status_code == 200:
      data = r.json()
      print(json.dumps(data, indent=2))
    else:
      print(f"ERROR {r.content}")

def buy_tao(quantity, price):
    PATH = '/api/v3/order'
    timestamp = int(time.time() * 1000)
    params = {
      'symbol': 'TAOUSDT',
      'side': 'BUY',
      'type': 'LIMIT',
      'quantity': quantity,
      'price': price,
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.post(url, headers=headers, params=params)
    if r.status_code == 200:
      data = r.json()
      print(json.dumps(data, indent=2))
    else:
      print(f"ERROR {r.content}")

def cancel_order(order_id):
    PATH = '/api/v3/order'
    timestamp = int(time.time() * 1000)
    params = {
      'symbol': 'TAOUSDT',
      'orderId': order_id,
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.delete(url, headers=headers, params=params)
    if r.status_code == 200:
      data = r.json()
      print(json.dumps(data, indent=2))
    else:
      print(f"ERROR {r.content}")
        
def cancel_all_orders():
    PATH = '/api/v3/openOrders'
    timestamp = int(time.time() * 1000)
    params = {
      'symbol': 'TAOUSDT',
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.delete(url, headers=headers, params=params)
    if r.status_code == 200:
      data = r.json()
      print(json.dumps(data, indent=2))
    else:
      print(f"ERROR {r.content}")

def get_bag():
    PATH = '/api/v3/account'
    timestamp = int(time.time() * 1000)
    params = {
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, headers=headers, params=params)
    if r.status_code == 200:
        data = r.json()
        return float(data['balances'][3]['free'])
    else:
      print(f"ERROR {r.content}")

def get_portefeuille():
    PATH = '/api/v3/account'
    timestamp = int(time.time() * 1000)
    params = {
      'recvWindow': 5000,
      'timestamp': timestamp
    }
    
    query_string = urlencode(params)
    params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, headers=headers, params=params)
    if r.status_code == 200:
        data = r.json()
        return float(data['balances'][0]['free'])
    else:
      print(f"ERROR {r.content}")

def get_trades(day):
  PATH = '/api/v3/myTrades'
  timestamp = int(time.time() * 1000)
  date_object = datetime.strptime(day, '%d.%m.%Y')
  start_time = int(date_object.timestamp())
  # end_time = int((date_object + timedelta(days=1)).timestamp()) - 1
  
  params = {
    'symbol' : 'TAOUSDT',
    'startTime' : start_time,
    'recvWindow': 5000,
    'timestamp': timestamp
  }
  
  query_string = urlencode(params)
  params['signature'] = hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
  url = urljoin(BASE_URL, PATH)
  r = requests.get(url, headers=headers, params=params)
  if r.status_code == 200:
      data = r.json()
      return data
  else:
    print(f"ERROR {r.content}")

# BotClass

In [7]:
class Bot:
  def __init__(self):
    self.porte_feuille = get_portefeuille()
    self.bag = get_bag()

    self.nb_time = 0

    self.prix_actuel = get_current_price()
    self.hist_prix3 = []

    self.last_trade = ""
    self.last_palier = round(get_current_price())
    self.reinv_dol = 170
    self.reinv_crypto = 2
    self.hist_portefeuille = []
    self.hist_bag = []

    # Modification possible : Nb de tao vendu et acheté
    self.tao_vente = 0.5
    self.tao_achat = 0.55

    # Modification possible : Palier haut (juste après changement de pente) et palier bas (vente après vente, etc...)
    self.palier_haut = 2
    self.palier_bas = 1

    cancel_all_orders()
    sell_tao(self.tao_vente, self.last_palier + self.palier_haut)
    buy_tao(self.tao_achat, self.last_palier - self.palier_haut)

  def achat_dol(self, dollars):
    self.porte_feuille -= dollars
    self.bag += dollars / self.prix_actuel
    print("ACHAT")

  def vente_dol(self, dollars):
    if(self.bag >= dollars / self.prix_actuel):
      self.porte_feuille += dollars
      self.bag -= dollars / self.prix_actuel
      print("VENTE")

  def achat_crypto(self, crypto):
    self.porte_feuille -= crypto * self.prix_actuel
    self.bag += crypto
    print("ACHAT")

  def vente_crypto(self, crypto):
    if(self.bag >= crypto):
      self.porte_feuille += crypto * self.prix_actuel
      self.bag -= crypto
      print("VENTE")

  def isRising(self):
    if(self.hist_prix3[0] > self.hist_prix3[-1]):
      return False
    if(self.hist_prix3[0] < self.hist_prix3[-1]):
      return True
    else:
      print("ERROR WITH RISING COEF")
      return None

  def running_bot(self):
    print(f"BOUCLE N#{self.nb_time}")
    print(f"prix actuel = {self.prix_actuel}")
    print(f"bag = {self.bag} / ({self.bag * self.prix_actuel} $)")
    print(f"portefeuille = {self.porte_feuille}")
    print(f"palier = {self.last_palier}")
    print(f"last trade = {self.last_trade}")

    self.hist_portefeuille.append(self.porte_feuille)
    self.hist_bag.append(self.bag)
    self.hist_prix3.append(self.prix_actuel)

    if(len(self.hist_prix3) > 3):
      self.hist_prix3 = self.hist_prix3[1:]

    # First orders check
    if(self.prix_actuel <= self.last_palier - self.palier_bas and self.last_trade == ""):
        cancel_all_orders()
        buy_tao(self.tao_achat, self.last_palier - self.palier_bas - self.palier_bas)
        sell_tao(self.tao_vente, self.last_palier - self.palier_bas + self.palier_haut)

        self.achat_crypto(self.tao_achat)
        self.last_trade = "achat"
        self.last_palier -= self.palier_bas

    elif(self.prix_actuel >= self.last_palier + self.palier_bas and self.last_trade == ""):
        cancel_all_orders()
        sell_tao(self.tao_vente, self.last_palier + self.palier_bas + self.palier_bas)
        buy_tao(self.tao_achat, self.last_palier - self.palier_haut + self.palier_bas)

        self.vente_crypto(self.tao_vente)
        self.last_trade = "vente"
        self.last_palier += self.palier_bas

    # Achat à 10
    if(self.isRising() == False and self.prix_actuel <= self.last_palier - self.palier_haut and (self.last_trade == "vente" or self.last_trade == "revente")):
        cancel_all_orders()
        buy_tao(self.tao_achat, self.last_palier - self.palier_haut - self.palier_bas)
        sell_tao(self.tao_vente, self.last_palier)

        self.achat_crypto(self.tao_achat)
        self.last_trade = "achat"
        self.last_palier -= self.palier_haut

    # Achat à 5
    elif(self.isRising() == False and self.prix_actuel <= self.last_palier - self.palier_bas and (self.last_trade == "achat" or self.last_trade == "reachat")):
        cancel_all_orders()
        buy_tao(self.tao_achat, self.last_palier - self.palier_bas - self.palier_bas)
        sell_tao(self.tao_vente, self.last_palier - self.palier_bas + self.palier_haut)

        self.achat_crypto(self.tao_achat)
        self.last_trade = "reachat"
        self.last_palier -= self.palier_bas

    # Vente à 10
    elif(self.isRising() == True and self.prix_actuel >= self.last_palier + self.palier_haut and (self.last_trade == "achat" or self.last_trade == "reachat")):
        cancel_all_orders()
        sell_tao(self.tao_vente, self.last_palier + self.palier_bas + self.palier_haut)
        buy_tao(self.tao_achat, self.last_palier)

        self.vente_crypto(self.tao_vente)
        self.last_trade = "vente"
        self.last_palier += self.palier_haut

    # Vente à 5
    elif(self.isRising() == True and self.prix_actuel >= self.last_palier + self.palier_bas and (self.last_trade == "vente" or self.last_trade == "revente")):
        cancel_all_orders()
        sell_tao(self.tao_vente, self.last_palier + self.palier_bas + self.palier_bas)
        buy_tao(self.tao_achat, self.last_palier - self.palier_haut + self.palier_bas)

        self.vente_crypto(self.tao_vente)
        self.last_trade = "revente"
        self.last_palier += self.palier_bas

    self.nb_time += 1
    self.prix_actuel = get_current_price()
    print("**********************")

In [ ]:
test = Bot()

while True:
    test.running_bot()
    time.sleep(60)

plt.plot(test.hist_bag)
plt.show

[
  {
    "symbol": "TAOUSDT",
    "orderId": "C01__348515450373607425",
    "price": "85.5",
    "origQty": "0.22",
    "type": "LIMIT",
    "side": "BUY",
    "executedQty": "0",
    "cummulativeQuoteQty": "0",
    "status": "NEW"
  },
  {
    "symbol": "TAOUSDT",
    "orderId": "C01__348515447848632323",
    "price": "88",
    "origQty": "0.2",
    "type": "LIMIT",
    "side": "SELL",
    "executedQty": "0",
    "cummulativeQuoteQty": "0",
    "status": "NEW"
  }
]
{
  "symbol": "TAOUSDT",
  "orderId": "C01__348517744725000193",
  "orderListId": -1,
  "price": "88.5",
  "origQty": "0.2",
  "type": "LIMIT",
  "side": "SELL",
  "transactTime": 1698875341355
}
{
  "symbol": "TAOUSDT",
  "orderId": "C01__348517748277563393",
  "orderListId": -1,
  "price": "87.5",
  "origQty": "0.22",
  "type": "LIMIT",
  "side": "BUY",
  "transactTime": 1698875342202
}
BOUCLE N#0
prix actuel = 87.818
bag = 12.07 / (1059.96326 $)
portefeuille = 2091.70829729
palier = 88
last trade = 
ERROR WITH RISING C